In [1]:
import numpy as np
import pandas as pd

import ast

import re
import unicodedata

In [2]:
raw_link = pd.read_csv('raw_link.csv')
raw_tthc = pd.read_csv('raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
(raw_tthc.link.str.split('ma_thu_tuc=').str[-1] == raw_tthc.maThuTuc).sum() == len(raw_tthc)

np.True_

In [4]:
raw_link.info()
raw_link.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9452 entries, 0 to 9451
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            9452 non-null   object
 1   phanLoai        9452 non-null   object
 2   boNganh         9452 non-null   object
 3   cauHoi          9452 non-null   object
 4   cauTraLoi       9449 non-null   object
 5   TTHCLienQuan    9452 non-null   object
 6   cauHoiLienQuan  9452 non-null   object
dtypes: object(7)
memory usage: 517.0+ KB


,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
4163,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,[],['Bộ Tài chính'],"Quy định về hồ sơ hải quan đối với ô tô, xe gắ...",Điều 75 Nghị định số 08/2015/NĐ-CP ngày 21/01/...,[],[]


In [5]:
raw_link = raw_link[~((raw_link['cauHoi']=='$questionInfo["CONTENT"]') | 
                    (raw_link['cauTraLoi']=='$i["CONTENT"]') |
                    (raw_link['cauHoi']).isna() |
                    (raw_link['cauTraLoi'].isna()) |
                    (raw_link['cauHoi'].apply(lambda x: len(x)<3 if type(x)==str else False)) |
                    (raw_link['cauTraLoi'].apply(lambda x: len(x)<3 if type(x)==str else False))
                    )] 
raw_link.shape

(9443, 7)

In [6]:
for col in ['phanLoai', 'boNganh', 'TTHCLienQuan', 'cauHoiLienQuan']:
    raw_link[col] = raw_link[col].apply(lambda x: str(sorted(ast.literal_eval(x))))
raw_link['cauHoi'] = raw_link['cauHoi'].str.strip()
raw_link['cauTraLoi'] = raw_link['cauTraLoi'].str.strip()

raw_link['cauHoiGoc'] = raw_link['cauHoi']
raw_link['cauTraLoiGoc'] = raw_link['cauTraLoi']

In [7]:
def clean_text(text):
    text = unicodedata.normalize('NFC', text)
    text = text.lower()
    # Remove characters that are not letters or spaces, including punctuation.
    cleaned_text = re.sub(r'[^a-z0-9àáạảãâầấậẩẫăắằẵẳặđèẹẻéẽêềếệểễìíịỉĩỵỳýỷỹóọỏõòôồốộổỗơợởỡớờùúụủũưừứựửữ/\s]+', ' ', text)
    # Trim extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

raw_link = raw_link.assign(cauHoi=raw_link['cauHoiGoc'].apply(clean_text))
raw_link = raw_link.assign(cauTraLoi=raw_link['cauTraLoiGoc'].apply(clean_text))

In [8]:
raw_link = raw_link.drop_duplicates(subset=['phanLoai', 'boNganh', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan'], keep='first')

raw_link['soPhanLoai'] = raw_link['phanLoai'].apply(ast.literal_eval).apply(len)
raw_link['soCauHoiLienQuan'] = raw_link['cauHoiLienQuan'].apply(ast.literal_eval).apply(len)
raw_link = raw_link.sort_values(by=['soCauHoiLienQuan', 'soPhanLoai'], ascending=False)
raw_link = raw_link.drop_duplicates(subset=['boNganh', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan'], keep='first')

raw_link['soBoNganh'] = raw_link['boNganh'].apply(ast.literal_eval).apply(len)
raw_link['soTTHCLienQuan'] = raw_link['TTHCLienQuan'].apply(ast.literal_eval).apply(len)
raw_link = raw_link.sort_values(by=['soBoNganh', 'soCauHoiLienQuan', 'soPhanLoai'], ascending=False)
raw_link = raw_link.drop_duplicates(subset=['cauHoi', 'cauTraLoi', 'TTHCLienQuan'], keep='first')

raw_link = raw_link.sort_values(by=['soTTHCLienQuan', 'soBoNganh', 'soCauHoiLienQuan', 'soPhanLoai'], ascending=False)
raw_link = raw_link.drop_duplicates(subset=['boNganh', 'cauHoi', 'cauTraLoi'], keep='first')
raw_link = raw_link.drop_duplicates(subset=['cauHoi', 'cauTraLoi'], keep='first')

raw_link = raw_link.reset_index(drop=True)
raw_link = raw_link.drop(columns=['cauHoi', 'cauTraLoi', 'soCauHoiLienQuan', 'soPhanLoai', 'soBoNganh', 'soTTHCLienQuan'])
raw_link = raw_link.rename(columns={'cauHoiGoc': 'cauHoi', 'cauTraLoiGoc': 'cauTraLoi'})
raw_link = raw_link[['link', 'phanLoai', 'boNganh', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan']]

In [9]:
raw_link.boNganh.apply(ast.literal_eval).apply(len).value_counts()

boNganh
1    5939
0    3350
Name: count, dtype: int64

In [10]:
raw_link.phanLoai.apply(ast.literal_eval).apply(len).value_counts()

phanLoai
0    7192
1    1136
2     572
3     389
Name: count, dtype: int64

In [11]:
raw_link.TTHCLienQuan.apply(ast.literal_eval).apply(len).value_counts()

TTHCLienQuan
1    3852
0    3161
2    1152
5     434
3     423
4     267
Name: count, dtype: int64

In [12]:
raw_link.cauHoiLienQuan.apply(ast.literal_eval).apply(len).value_counts()

cauHoiLienQuan
0     3571
2      818
1      645
3      499
4      411
5      316
9      294
6      266
10     261
7      245
8      188
13     158
11     148
12     139
14     115
20     102
16      87
21      83
19      74
29      68
45      64
24      61
17      59
15      57
23      54
22      50
36      49
28      48
47      44
18      42
31      38
42      37
33      35
27      35
30      24
25      20
38      17
26      14
32       9
34       8
35       7
37       7
39       4
56       4
48       4
40       4
51       2
41       1
87       1
63       1
50       1
Name: count, dtype: int64

In [13]:
for col in ['phanLoai', 'boNganh', 'TTHCLienQuan', 'cauHoiLienQuan']:
    raw_link[col] = raw_link[col].apply(ast.literal_eval)

In [14]:
raw_link['boNganh'] = raw_link['boNganh'].apply(lambda x: x[0] if len(x)>0 else '')

In [15]:
raw_link.info()
raw_link.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9289 entries, 0 to 9288
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            9289 non-null   object
 1   phanLoai        9289 non-null   object
 2   boNganh         9289 non-null   object
 3   cauHoi          9289 non-null   object
 4   cauTraLoi       9289 non-null   object
 5   TTHCLienQuan    9289 non-null   object
 6   cauHoiLienQuan  9289 non-null   object
dtypes: object(7)
memory usage: 508.1+ KB


,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
8425,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,[],,Cá nhân nước ngoài chuyển đổi chứng chỉ hành n...,Theo quy định tại Khoản 37 Điều 1 Nghị định số...,[],[]


In [16]:
raw_link.to_csv('preprocess_1.csv', index=False)